In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor

In [2]:
df = pd.read_csv("Removed_outliers_byGroup_data.csv")
df

,SHLT,BMI,MSTOT,COGTOT,INHPFN,HHHRES,HCHILD,LIVSIB,HINPOV,HAIRA,HATOTB,IEARN,HITOT,PENINC,HIGOV,PRPCNT,SLFEMP,RETMON
0,5.0,33.0,14.0,17.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,20000.0,22400.0,0.0,0.0,0.0,0.0,0
1,4.0,23.8,8.0,14.0,0.0,2.0,6.0,2.0,0.0,0.0,15000.0,25000.0,107000.0,0.0,0.0,1.0,0.0,0
2,3.0,26.0,15.0,27.0,0.0,2.0,2.0,1.0,0.0,40000.0,290000.0,103000.0,134384.0,0.0,0.0,1.0,0.0,0
3,4.0,40.7,11.0,16.0,0.0,3.0,4.0,7.0,0.0,0.0,16477.0,62000.0,72157.0,0.0,0.0,0.0,0.0,0
4,3.0,22.8,15.0,31.0,0.0,4.0,4.0,4.0,0.0,4000.0,138300.0,15000.0,95660.0,0.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33533,4.0,39.6,14.0,23.0,0.0,2.0,5.0,6.0,1.0,0.0,27500.0,27000.0,29112.0,0.0,0.0,0.0,0.0,0
33534,1.0,18.4,14.0,27.0,0.0,5.0,6.0,5.0,1.0,0.0,90000.0,0.0,0.0,0.0,0.0,1.0,0.0,0
33535,1.0,26.9,9.0,25.0,0.0,2.0,7.0,7.0,1.0,0.0,2341.0,3295.0,3295.0,0.0,0.0,1.0,0.0,0
33536,4.0,29.3,13.0,23.0,0.0,3.0,3.0,7.0,1.0,0.0,0.0,18000.0,18000.0,0.0,0.0,0.0,0.0,0


In [3]:
target_features = ['SHLT', 'COGTOT', 'MSTOT']
continuous_features = ['BMI', 'INHPFN',  'HHHRES', 'HCHILD','LIVSIB',  'HAIRA', 'HATOTB', 'IEARN','HITOT', 'PRPCNT']
X = df.drop(target_features, axis = 1).values
y = df[['SHLT', 'COGTOT', 'MSTOT']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [4]:
regr = MLPRegressor(random_state=45, max_iter=500).fit(X_train, y_train)
regr.score(X_test, y_test)
mean_squared_error(y_test, regr.predict(X_test))

265301.2798924823

In [5]:
for i in range(1,11,1):
    neigh = KNeighborsRegressor(n_neighbors=i)
    neigh.fit(X_train, y_train)
    print("k=",i,neigh.score(X_test, y_test), mean_squared_error(y_test, neigh.predict(X_test)))

k= 1 0.4970545495454652 3.179805207712185
k= 2 0.22140246941217176 4.9284535877559135
k= 3 0.09400665984165983 5.716460169173236
k= 4 0.08669164494767567 5.740203239912542
k= 5 0.07305873962797264 5.830470284237716
k= 6 0.07215737348787037 5.864630402614775
k= 7 0.07221234441007192 5.870322368661242
k= 8 0.07045815682515932 5.883932120850726
k= 9 0.07165007305230715 5.869193224231931
k= 10 0.07413449922944222 5.841993241900204


In [7]:
kmeans = KMeans(n_clusters=50, random_state=42).fit(X_train)
centers = kmeans.cluster_centers_
distances_train = np.array([np.linalg.norm(X_train - center, axis=1) for center in centers]).T
distances_test = np.array([np.linalg.norm(X_test - center, axis=1) for center in centers]).T
lr_model = LinearRegression().fit(distances_train, y_train)
lr_model.score(distances_test, y_test)
mean_squared_error(y_test, lr_model.predict(distances_test))

5.797429451667242

In [8]:
lr_model = LinearRegression().fit(X_train, y_train)
lr_model.score(X_test, y_test)
mean_squared_error(y_test, lr_model.predict(X_test))

5.754482373904952

In [9]:
rf = RandomForestRegressor(n_jobs=-1).fit(X_train,y_train)

rf.score(X_test,y_test)
mean_squared_error(y_test, rf.predict(X_test))

2.184281733253822